In [443]:
import pandas as pd
import numpy as np
import psycopg2 as pg
import configparser
import os

In [444]:
#Path pre_corte 

path_pre_corte = r'F:\Qualidade_Florestal\01- SÃO PAULO\07- Monitoramentos de Silvicultura\06 - SOP CODE\01 - Bases de Dados\01 - Bases Legado\sop_legado_pre_corte.xlsx'

In [445]:
#Path credenciais
crendenciais_path = r'\\glwfs02.lwart.net\LWC-FLORESTAL\Qualidade_Florestal\03- ADMINISTRATIVO\2023\06- COLABORADORES\Gabriel\18 - SQL\sets.conf'

credenciais = configparser.ConfigParser()

credenciais.read(crendenciais_path)

logins = credenciais['informacoes']



conn_dwh = pg.connect(
    user = logins['login'],
    password = logins['senha'],
    host = logins['host'],
    port = logins['port'],
    database = logins['data_base']
    )

In [446]:
#Função máscara

def mascara(df, coluna,n_digitos):
    df[coluna] = df[coluna].astype(str)
    df[coluna] = str("000") + df[coluna]
    df[coluna] = df[coluna].astype(str).str[-n_digitos:] 
    return df

In [447]:
#Realizando conexão do banco -> cadastro histórico

cursor = conn_dwh.cursor()

#Se atentar a região de São paulo que é 1 

#Necessário mudar a consulta para fazer um left join na consulta d_cfg_espacamento para trazer a informação de espaçamento do cadastro e nome do projeto, logo foi necessário ter duas consultas


cursor.execute("""
SELECT 
    f.data_plantio,
    f.cd_regiao,
    f.id_projeto,
    f.cd_projeto,
    f.cd_talhao,
    f.num_ciclo,
    f.cd_material_genetico,
    f.cd_regime,
    f.num_rotacao,
    f.vlr_area,
    f.source,
    f.data_reg,
    f.est_reg,
    f.cd_uso_solo,
    f.cd_caracteristica_uso_solo,
    f.cd_espacamento,
    d.dcr_espacamento,
    p.nom_projeto
FROM 
    forestry.f_cfg_uso_solo f
LEFT JOIN 
    forestry.d_cfg_espacamento d
ON 
    f.cd_espacamento = d.cd_espacamento
    
LEFT JOIN 
    forestry.cfg_projeto p

ON
    f.id_projeto = p.id_projeto
    
WHERE 
    f.source = 'BSP' 
    AND p.source = 'BSP'
    AND p.cd_regiao = '1'
    AND p.est_reg = 'A'
    AND f.tip_reg = 'A'  
    AND f.cd_regiao IN ('1','2')
    
    
    
""")



dados_consulta = cursor.fetchall()

cursor.close()

#Manter a conexão do dwh aberta
#conn_dwh.close() A conexão não foi fechada pois será utilizada novamente na consulta abaixo

colunas_cad = [desc[0] for desc in cursor.description]


cadastro_ciclo = pd.DataFrame(dados_consulta,columns=colunas_cad)

In [448]:
pre_corte_legado = pd.read_excel(path_pre_corte, sheet_name='bd_precorte', engine = 'openpyxl', dtype={'Talhão':'str', 'Id Projeto':'str'})

In [449]:
pre_corte_legado = mascara(pre_corte_legado,'Talhão', 3)

pre_corte_legado = mascara(pre_corte_legado,'Id Projeto', 4)

pre_corte_legado['objeto de locação'] = pre_corte_legado['Id Projeto'] + pre_corte_legado['Talhão']


In [450]:
len_pre_corte_legado = len(pre_corte_legado)
len_pre_corte_legado

obj_inicio = set(pre_corte_legado['objeto de locação'])


In [451]:
cadastro_ciclo = mascara(cadastro_ciclo,'cd_talhao',3)
cadastro_ciclo = mascara(cadastro_ciclo,'id_projeto',4)
cadastro_ciclo['objeto de locação'] = cadastro_ciclo['id_projeto'] + cadastro_ciclo['cd_talhao']

In [452]:
cadastro_ciclo = cadastro_ciclo[['objeto de locação','num_ciclo','cd_regime','data_reg','est_reg','cd_uso_solo']]
cadastro_ciclo

,objeto de locação,num_ciclo,cd_regime,data_reg,est_reg,cd_uso_solo
0,0755011,1,7,2020-03-19 12:48:00,H,631
1,0755012,1,7,2020-03-19 12:48:00,H,632
2,0755013,1,7,2020-03-19 12:48:00,H,633
3,0755014,1,7,2020-03-19 12:48:00,H,634
4,0755015,1,7,2020-03-19 12:48:00,H,635
...,...,...,...,...,...,...
43430,0511021,2,6,2023-06-30 14:04:41,A,90433
43431,0511520,None,None,2023-06-30 14:05:16,H,90434
43432,0511307,2,6,2023-06-30 14:05:23,A,90435
43433,0166025,1,5,2020-01-04 12:53:40,H,19307


In [453]:
pre_corte_legado = pre_corte_legado.merge(cadastro_ciclo, left_on='objeto de locação', right_on='objeto de locação', how='left')

In [454]:
#Removendo talhões em branco do pre corte legado e vendo o tamanho

#Quando não tiver data de pre corte, devo trazer o cto

pre_corte_legado['Data PréCorte_check'] = np.where(pd.isna(pre_corte_legado['Data PréCorte']), pre_corte_legado['CTO Derrubada'], pre_corte_legado['Data PréCorte'])

pre_corte_legado = pre_corte_legado[pd.notna(pre_corte_legado['Data PréCorte_check'])]

len(pre_corte_legado)

10167

In [455]:
pre_corte_legado['delta registro'] = (pre_corte_legado['Data PréCorte_check'] - pre_corte_legado['data_reg']).dt.days


In [456]:
pre_corte_legado['n de ocorrencias'] = pre_corte_legado.groupby(['objeto de locação'])['objeto de locação'].transform('count')

In [457]:
#Intervalo definido, mantendo somente a operação que foi feita dentro do intervalo max e min

intervalo_max = 365
intervalo_min = -365


#Só posso fazer essa regra se o talhão for maior que 1

pre_corte_legado.loc[(pre_corte_legado['delta registro'] <= intervalo_max) & 
                     (pre_corte_legado['delta registro'] >= intervalo_min) & 
                     (pre_corte_legado['n de ocorrencias'] > 2), 'n de ocorrencias'] = \
pre_corte_legado[(pre_corte_legado['delta registro'] <= intervalo_max) & 
                 (pre_corte_legado['delta registro'] >= intervalo_min)]['n de ocorrencias']



In [458]:
pre_corte_legado = pre_corte_legado.sort_values(['objeto de locação','delta registro','Data PréCorte_check', 'data_reg'], ascending=[True,False,True,True])

In [459]:
pre_corte_legado = pre_corte_legado.drop_duplicates(subset='objeto de locação' ,keep='first')

In [460]:
obj_final = set(pre_corte_legado['objeto de locação'])

In [461]:
delta_obj = obj_inicio - obj_final

delta_obj

set()

In [462]:
tamanho_pre_corte_legado - len(pre_corte_legado)

#A diferença fica somente para os talhões que eram duplicados

65

In [463]:
pre_corte_legado = pre_corte_legado.drop(['n de ocorrencias', 'Data PréCorte_check','delta registro'],axis=1)

In [441]:
path_save = r'F:\Qualidade_Florestal\01- SÃO PAULO\07- Monitoramentos de Silvicultura\06 - SOP CODE\01 - Bases de Dados\01 - Bases Legado'

In [465]:
pre_corte_legado.to_excel(os.path.join(path_save,'sop_legado_pre_corte_cd_uso_solo.xlsx'))